## Please Download the gguf file (model_path)

download this: Llama-3.2-1B-Instruct-IQ3_M.gguf
link: https://huggingface.co/bartowski/Llama-3.2-1B-Instruct-GGUF/blob/main/Llama-3.2-1B-Instruct-IQ3_M.gguf

Install some dependencies

In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U datasets==2.17.0
!pip install -q -U transformers==4.38.1
!pip install langchain sentence-transformers chromadb langchainhub

!pip install langchain-community langchain-core

Get the Model You Want

In [ ]:
#!pip install --upgrade --upgrade-strategy eager "optimum[neural-compressor]"
!pip install llama-cpp-python

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb

import time
import uuid
from llama_cpp import Llama
#from fuzzywuzzy import fuzz
#from optimum.quantization import QuantizeruenceClassification


Define Variables

In [ ]:
from llama_cpp import Llama

model_path = "./Llama-3.2-1B-Instruct-IQ3_M.gguf"

model = Llama(model_path=model_path, n_ctx=2048, n_threads=8)


In [ ]:
ls -lh /content/Llama-3.2-1B-Instruct-IQ3_M.gguf


Define Data Sources

In [12]:
import pandas as pd

file_names = [
    "study_permit_general",
    "work_permit_student_general",
    "work-study-data-llm",
    "vancouver_transit_qa_pairs",
    "permanent_residence_student_general",
    "data-with-sources"
]

all_texts = []

for file in file_names:
    path = f'./sample_data/{file}.csv'
    try:
        df = pd.read_csv(path)
        df.columns = df.columns.str.lower()

        if 'question' in df.columns and 'answer' in df.columns:
            df = df.drop_duplicates(subset=['question'])
            df['text'] = df['question'].fillna('') + ' ' + df['answer'].fillna('')
        else:
            print(f"no text columns in {file}")
            continue
        all_texts.extend(df['text'].tolist())
    except Exception as e:
        print(f"Error loading {file}: {e}")


Set Embedding Model, and Chroma Client to Interact w Vector Database and Create Collections

In [14]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import uuid
import pandas as pd
#from fuzzywuzzy import fuzz

# pt model for generating embeddings used pretty often
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-MiniLM-L6-v2"
)

# persistent client to interact w chroma vector store
client = chromadb.PersistentClient(path="./chroma_db")

# create collections for each data (for testing rn)
collection = client.get_or_create_collection(name="combined_docs")

# seems like better results if we remove duplicates and very similar data
data = pd.DataFrame({"text": all_texts})
data = data.drop_duplicates()
all_texts = data["text"].tolist()

print(f"successfully added {len(all_texts)} documents to Chroma DB.")

successfully added 442 documents to Chroma DB.


Function to add data to collection by embedding them

In [15]:
def add_data_to_collection_batch(collection, texts, batch_size=3):
    for idx in range(0, len(texts), batch_size):
        try:
            batch_texts = texts[idx: idx + batch_size]

            embeddings = embedding_model.embed_documents(batch_texts)

            batch_ids = [str(uuid.uuid4()) for _ in batch_texts]

            collection.add(
                ids=batch_ids,
                embeddings=embeddings,
                documents=batch_texts
            )
            print(f"successfully added {len(batch_texts)} documents (Batch {idx}-{idx + batch_size - 1})")
        except Exception as e:
            print(f"Error processing batch starting at index {idx}: {e}")

In [ ]:
add_data_to_collection_batch(collection, all_texts)
print(f"successfully added {len(all_texts)} documents to the Chroma collection.")

Function to now match for releveant document

In [17]:
def get_relevant_documents(query, n_results=3):
    try:
        query_embeddings = embedding_model.embed_documents([query])[0]

        results = collection.query(query_embeddings=[query_embeddings], n_results=n_results)
        print(f"Query Results: {results}")

        return results['documents'][0] if results['documents'] else []
    except Exception as e:
        print(f"Error querying: {e}")
        return []

Generate Answer

In [18]:
def generate_answer(query):
    response_before_rag = model(query, max_tokens=200, temperature=0.1)["choices"][0]["text"]

    relevant_documents = get_relevant_documents(query)
    if not relevant_documents:
        return {
            "Before RAG Response": response_before_rag,
            "After RAG Response": "Sorry, no relevant documents found."
        }

    relevant_texts = "\n\n".join(relevant_documents)
    rag_prompt = f"""
    You are a helpful assistant for international students. Here are relevant documents:

    {relevant_texts}

    Please respond to the following question based on the documents above. Be conversational but concise:

    Question: {query}

    Answer:
    """

    response_after_rag = model(rag_prompt, max_tokens=300, temperature=0.1)["choices"][0]["text"]

    return {
        "Before RAG Response": response_before_rag,
        "After RAG Response": response_after_rag
    }


Example Usage

In [19]:
test_queries = [
    "How do I apply for a study permit in Canada?",
    "Can I work while studying on a student visa?",
    "What happens if my study permit expires before I finish my program?",
    "Do I need a new study permit if I change schools?",
    "How long does it take to process a Canadian study permit?",
    "Am I allowed to work off-campus as an international student?",
    "How many hours can I work while studying in Canada?",
    "What documents do I need to apply for a co-op work permit?",
    "Can I work in Canada after I graduate?",
    "What is a Post-Graduation Work Permit (PGWP) and how do I apply?",
    "How do I apply for MSP (Medical Services Plan) in British Columbia?",
    "Is MSP mandatory for international students?",
    "What healthcare services are covered under MSP?",
    "What should I do if I get sick and don’t have insurance yet?",
    "Can I use private health insurance instead of MSP?",
    "What are my options for student housing in Vancouver?",
    "How much does rent typically cost for international students?",
    "What should I check before signing a lease in Canada?",
    "Are there any student discounts for accommodation?",
    "How can I find a roommate in Canada?",
    "How do I open a bank account as an international student?",
    "What documents do I need to get a student bank account?",
    "Can I get a credit card as an international student?",
    "How do I send money to my home country from Canada?",
    "What scholarships are available for international students?",
    "How does the Compass Card work for transit in Vancouver?",
    "Am I eligible for a U-Pass as an international student?",
    "What is the best way to get around Vancouver on a budget?",
    "Where can I find the bus and SkyTrain schedules?",
    "Are there student discounts for public transportation?",
    "Can I apply for permanent residence after graduating?",
    "What is the Canadian Experience Class (CEC) immigration program?",
    "How can I improve my chances of getting permanent residence?",
    "What are the eligibility requirements for Express Entry?",
    "Does having a Canadian degree help with PR applications?"
]

for idx, user_query in enumerate(test_queries, start=1):
    responses = generate_answer(user_query)

    print("\n" + "="*50)
    print(f"Test Query {idx}: {user_query}")
    print("="*50)
    print("Response Before RAG:\n", responses["Before RAG Response"])
    print("\nResponse After RAG:\n", responses["After RAG Response"])
    print("="*50 + "\n\n")

#add_all_texts_to_collection()

# big speedup


llama_perf_context_print:        load time =    1100.22 ms
llama_perf_context_print: prompt eval time =    1099.32 ms /    12 tokens (   91.61 ms per token,    10.92 tokens per second)
llama_perf_context_print:        eval time =    5062.57 ms /   199 runs   (   25.44 ms per token,    39.31 tokens per second)
llama_perf_context_print:       total time =    6212.13 ms /   211 tokens


Query Results: {'ids': [['053f6297-87e3-452c-b615-4cac8daeb372', 'f138a4b0-feef-4818-a5a3-3a2d0fa6d90b', '092633c2-981f-43dc-a318-710b5c2fdac7']], 'embeddings': None, 'documents': [['How do I apply for a study permit?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=479&top=15") You should apply for a study permit\xa0before coming to Canada. Make sure\xa0you have a letter of acceptance\xa0from a\xa0designated learning institution("https://www.canada.ca/en/services/immigration-citizenship/helpcentre/glossary.html#designated_learning_institution")\xa0in Canada\xa0before you apply.\nYou should\xa0apply online("https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/study-permit/apply.html")\xa0for a study permit, whether you’re outside of Canada or already in Canada. If you\xa0can’t\xa0apply online because of a disability or because there’s a\xa0\xa0problem with the online application("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=479&top=15#i

Llama.generate: 1 prefix-match hit, remaining 790 prompt tokens to eval
llama_perf_context_print:        load time =    1100.22 ms
llama_perf_context_print: prompt eval time =    3292.89 ms /   790 tokens (    4.17 ms per token,   239.91 tokens per second)
llama_perf_context_print:        eval time =    3101.36 ms /   115 runs   (   26.97 ms per token,    37.08 tokens per second)
llama_perf_context_print:       total time =    6418.82 ms /   905 tokens
Llama.generate: 1 prefix-match hit, remaining 10 prompt tokens to eval



Test Query 1: How do I apply for a study permit in Canada?
Response Before RAG:
  The process typically involves the following steps:
1. Check if you need a study permit: Determine if you need a study permit to study in Canada. You can check the Immigration, Refugees and Citizenship Canada (IRCC) website to see if you qualify.
2. Gather required documents: Gather the necessary documents, including:
	* A valid passport
	* A completed application form (IRCST-1)
	* Proof of language proficiency (e.g., English or French language test results)
	* Proof of sufficient funds to support yourself during your stay in Canada
	* Proof of health insurance
	* Police certificate (if required)
3. Submit your application: Submit your application online through the IRCC website or by mail. You can also submit your application by mail, but it's recommended to use the online option.
4. Pay the application fee: The application fee for a study permit is currently $150 CAD.
5. Wait for your application to be

llama_perf_context_print:        load time =    1100.22 ms
llama_perf_context_print: prompt eval time =     169.38 ms /    10 tokens (   16.94 ms per token,    59.04 tokens per second)
llama_perf_context_print:        eval time =    4619.87 ms /   199 runs   (   23.22 ms per token,    43.07 tokens per second)
llama_perf_context_print:       total time =    4835.76 ms /   209 tokens


...


Test Query 34: What are the eligibility requirements for Express Entry?
Response Before RAG:
  The eligibility requirements for Express Entry vary depending on the category of the candidate. Here are some of the key eligibility requirements for different categories:

**Federal Skilled Work Experience (FSEW) Category:**

* A minimum of 2 years of work experience in a skilled occupation
* A minimum of $30,000 in gross income in the past 2 years
* A minimum of a bachelor's degree or higher
* A minimum of a 30% increase in the previous year's income
* A minimum of a 30% increase in the previous year's education level

**Federal Skilled Worker Category:**

* A minimum of 2 years of work experience in a skilled occupation
* A minimum of $30,000 in gross income in the past 2 years
* A minimum of a bachelor's degree or higher
* A minimum of a 30% increase in the previous year's income
* A minimum of a 30% increase in the previous year's

Response After RAG:
  Express Entry is a federal system

llama_perf_context_print:        load time =    1100.22 ms
llama_perf_context_print: prompt eval time =     562.43 ms /    10 tokens (   56.24 ms per token,    17.78 tokens per second)
llama_perf_context_print:        eval time =    4527.23 ms /   199 runs   (   22.75 ms per token,    43.96 tokens per second)
llama_perf_context_print:       total time =    5136.29 ms /   209 tokens
Llama.generate: 1 prefix-match hit, remaining 797 prompt tokens to eval


Query Results: {'ids': [['e553dc11-9621-4fa4-acee-0b0530f320dc', '81cc04f5-495f-4289-90c1-3ded48a64402', 'b17b2669-83da-4d66-acc4-0e4d7a09325c']], 'embeddings': None, 'documents': [['I’m an international student in Canada. Can I work while studying?("https://ircc.canada.ca/english/helpcentre/answer.asp?qnum=495&top=15") You may be eligible to work in Canada while studying, if your study permit includes a condition that says you can work on or off campus. You must also meet all the other requirements. You can only start working in Canada when you start your study program. You can’t work before your studies begin.\n\nHowever work as a co-op Student or intern: You will need to get a work permit if your study program includes a required co-op or internship placement.\xa0Find out how to apply for a co-op work permit("https://www.canada.ca/en/immigration-refugees-citizenship/services/study-canada/work/intern.html").\n\nAnd for Work after you graduate: You may also be able to apply for a post

llama_perf_context_print:        load time =    1100.22 ms
llama_perf_context_print: prompt eval time =    3059.07 ms /   797 tokens (    3.84 ms per token,   260.54 tokens per second)
llama_perf_context_print:        eval time =    1576.18 ms /    63 runs   (   25.02 ms per token,    39.97 tokens per second)
llama_perf_context_print:       total time =    4648.71 ms /   860 tokens



Test Query 35: Does having a Canadian degree help with PR applications?
Response Before RAG:
 ?
Having a Canadian degree can be beneficial when it comes to PR applications, but it's not a guarantee of success. Here's why:

1. **Reputation**: A Canadian degree is considered highly prestigious, which can give you an edge when it comes to PR applications. Employers may view you as a more desirable candidate with a Canadian degree.
2. **Recognition**: Canadian degrees are recognized globally, which can make it easier to transfer credits or use a Canadian degree to demonstrate your qualifications to potential employers.
3. **Work experience**: Having a Canadian degree can also give you a head start in terms of work experience. Many Canadian employers may prefer to hire candidates with a degree from a reputable university.
4. **Networking**: Having a Canadian degree can also open doors to networking opportunities, as you may be more likely to have connections in the industry.

However, ther